# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [28]:
import pandas as pd
import numpy as np
from scipy import stats

In [29]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [30]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [31]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [32]:
data.columns

Index(['id', 'ad', 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer',
       'military', 'empholes', 'occupspecific', 'occupbroad', 'workinschool',
       'email', 'computerskills', 'specialskills', 'firstname', 'sex', 'race',
       'h', 'l', 'call', 'city', 'kind', 'adid', 'fracblack', 'fracwhite',
       'lmedhhinc', 'fracdropout', 'fraccolp', 'linc', 'col', 'expminreq',
       'schoolreq', 'eoe', 'parent_sales', 'parent_emp', 'branch_sales',
       'branch_emp', 'fed', 'fracblack_empzip', 'fracwhite_empzip',
       'lmedhhinc_empzip', 'fracdropout_empzip', 'fraccolp_empzip',
       'linc_empzip', 'manager', 'supervisor', 'secretary', 'offsupport',
       'salesrep', 'retailsales', 'req', 'expreq', 'comreq', 'educreq',
       'compreq', 'orgreq', 'manuf', 'transcom', 'bankreal', 'trade',
       'busservice', 'othservice', 'missind', 'ownership'],
      dtype='object')

In [33]:
race = data.race
call = data.call

In [34]:
data.race.count()

4870

In [35]:
print(data[race == 'b'].race.count())
print(data[race == 'w'].race.count())

2435
2435


In [36]:
call_b = data[race == 'b'].call
call_w = data[race == 'w'].call

### What test is appropriate for this problem? Does CLT apply?

For this test a binomial distrubtion will be used to test the chances of a call back becuase of the categorical values being used.

### What are the null and alternate hypotheses?
Peasron's Chis Squared test should be used to test whether or not a person's race independent to whether or not they receive a callback or not.

In [37]:
df = pd.DataFrame([data.race, data.call==1.0], ['race', 'call']).transpose()
df['values'] = 1
contingency_table = pd.pivot_table(df, values='values', index='race', columns=['call'], aggfunc=np.sum)
contingency_table

call,False,True
race,,
b,2278,157
w,2200,235


In [41]:
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
print(chi2)
print(p)
print(dof)
print(expected)

16.4490285842
4.99757838996e-05
1
[[ 2239.   196.]
 [ 2239.   196.]]


Because the null hypothesis is below the level of significance or 0.05, the null hypothesis is rejected.

In [42]:
def summarize(x):
    mu, s, n = x.mean(), x.std(), len(x)
    sem = stats.sem(x)
    CI = stats.norm.interval(0.90, mu, sem)
    print ('Sample mean:             {mu:0.2f}\n'
           'Sample variance:         {v:0.3f}\n'
           'Sample margin of error:  {sem:0.4f}\n'
           '90% confidence interval: {CI}'
           .format(mu=mu, v=s**2, sem=sem, CI=CI))

### Compute margin of error, confidence interval, and p-value.

In [43]:
summarize(call_b)

Sample mean:             0.06
Sample variance:         0.060
Sample margin of error:  0.0050
90% confidence interval: (0.05628806842760025, 0.072664702997646935)


In [44]:
summarize(call_w)

Sample mean:             0.10
Sample variance:         0.087
Sample margin of error:  0.0060
90% confidence interval: (0.086664295855603618, 0.10635418527976473)


### Write a story describing the statistical significance in the context or the original problem

Becasue the null hypothesis that race and receiving a call back are not independent of eachother was rejected it can be said that there is evidence showing companies make hiring desicions based on race. To avoid this issue, all personal information should be removed and only professional information should be present on a resume.

### Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

This analysis is not detailed enough to make such a claim because race is not the only personal information that can be tested. Other information to test should be gender, education, and hometown.